In [1]:
import os
from torch.utils.data import Dataset, DataLoader, ConcatDataset,  WeightedRandomSampler
from skimage import io, transform
from torchvision import transforms, utils
import torch
import numpy as np
import nibabel as nib
from random import randint
from PIL import Image
import torch.optim as optim
import time
import torch.nn as nn
from my_Dataset import TrainDataset
import matplotlib.pyplot as plt
from weight_axial import weight,count
# import res18 as Res
import Resnet as Res
# from QN import QuickNat

In [2]:
gpu_id = 0
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_id)

device = torch.device('cuda')
print(device)

cuda


In [3]:
sub_idx = 0
slice_idx = 2
T1a_dir = '/home/xiaoyu/MRIdata_group/T1w/axial/sub{}/slice{}'.format(sub_idx,slice_idx)
brainmask_dir = '/home/xiaoyu/MRIdata_group/brainmask/axial/sub{}/group{}'.format(sub_idx,slice_idx)
total_data = TrainDataset(T1a_dir=T1a_dir, brainmask_dir = brainmask_dir)

In [4]:
for sub_idx in range(1,330):
    T1a_dir = '/home/xiaoyu/MRIdata_group/T1w/axial/sub{}/slice{}'.format(sub_idx,slice_idx)
    brainmask_dir = '/home/xiaoyu/MRIdata_group/brainmask/axial/sub{}/group{}'.format(sub_idx,slice_idx)
    train_data = TrainDataset(T1a_dir=T1a_dir, brainmask_dir = brainmask_dir)
    total_data = total_data + train_data
print(len(total_data))

3300


In [ ]:
dataloader = DataLoader(total_data, batch_size = 12, shuffle = True, num_workers = 4)
print(len(dataloader))

275


In [ ]:
# total_data[0]['T1a'].shape

In [ ]:
# params= {'num_channels':1, 'num_filters':64, 'kernel_h':5, 'kernel_w':5, 'kernel_c':1, 'stride_conv':1,'pool':2, 
#          'stride_pool':2, 'num_class':3, 'se_block': 'CSSE','drop_out':0.2}
# model = QuickNat(params)
model = Res.resnet18()
start=time.time()
nb_param=0
for param in model.parameters():
    nb_param+=np.prod(list(param.data.size()))
print(nb_param)

11169216


In [ ]:
# print(model)

In [ ]:
bs=5
x=torch.rand(bs,1,128,128)
y = model(x)
print(y.size())

torch.Size([5, 3, 128, 128])


In [ ]:
model = model.to(device)
weight = weight.to(device)
criterion = nn.NLLLoss(weight = weight)
optimizer = optim.Adam(model.parameters() ,lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.85)

In [ ]:
loss_list = []
for epoch in range(0,200):
    running_loss = 0
    num_batches = 0
    scheduler.step()
    for i_batch, sample_batched in enumerate(dataloader):
    
        optimizer.zero_grad()
        
        #get the inputs
        inputs, labels = sample_batched['T1a'], sample_batched['brainmask']
        

        inputs = inputs.to(device)
        labels = labels.to(device)

        inputs.requires_grad_()
        
        #forward + backward +optimize
        scores = model(inputs)

          
        # Define the loss
        loss = criterion(scores, labels.long()) 
        loss.backward()
        optimizer.step()
        
        # compute and accumulate stats
        running_loss += loss.detach().item()

       
        num_batches+=1 
        
        
    # AVERAGE STATS THEN DISPLAY    
    total_loss = running_loss/num_batches
    loss_list = np.append(loss_list,total_loss)
   
    elapsed = (time.time()-start)/60
        
    print('epoch=',epoch, '\t time=', elapsed,'min', '\t loss=', total_loss ) 
    if epoch%10==0:
        print(loss_list)
        torch.save(model.state_dict(),'model')
   
 
print('Finish Training')

epoch= 0 	 time= 0.581281328201294 min 	 loss= 0.5971902255578475
[0.59719023]
epoch= 1 	 time= 1.0605087955792745 min 	 loss= 0.5347271073948253
epoch= 2 	 time= 1.5443926493326823 min 	 loss= 0.5171541708165949
epoch= 3 	 time= 2.0248178283373517 min 	 loss= 0.5065172930197283
epoch= 4 	 time= 2.5096007148424784 min 	 loss= 0.4941827379573475
epoch= 5 	 time= 2.9951432943344116 min 	 loss= 0.4801149401881478
epoch= 6 	 time= 3.481102482477824 min 	 loss= 0.46295833099972117
epoch= 7 	 time= 3.9670823574066163 min 	 loss= 0.44526272361928765
epoch= 8 	 time= 4.448407824834188 min 	 loss= 0.4193103337287903
epoch= 9 	 time= 4.929426741600037 min 	 loss= 0.39255718978968535
epoch= 10 	 time= 5.411434626579284 min 	 loss= 0.36064660386605696
[0.59719023 0.53472711 0.51715417 0.50651729 0.49418274 0.48011494
 0.46295833 0.44526272 0.41931033 0.39255719 0.3606466 ]
epoch= 11 	 time= 5.894329873720805 min 	 loss= 0.3280150404843417
epoch= 12 	 time= 6.379026067256928 min 	 loss= 0.301357720

epoch= 74 	 time= 36.42112883726756 min 	 loss= 0.15346411328424106
epoch= 75 	 time= 36.90603865385056 min 	 loss= 0.14224684568968687
epoch= 76 	 time= 37.39109570980072 min 	 loss= 0.13660005534237082
epoch= 77 	 time= 37.87422842184703 min 	 loss= 0.13457968844608828
epoch= 78 	 time= 38.35893648862839 min 	 loss= 0.13344878223809328
epoch= 79 	 time= 38.843806322415666 min 	 loss= 0.13309995968233454
epoch= 80 	 time= 39.32785982290904 min 	 loss= 0.1328540733998472
[0.59719023 0.53472711 0.51715417 0.50651729 0.49418274 0.48011494
 0.46295833 0.44526272 0.41931033 0.39255719 0.3606466  0.32801504
 0.30135772 0.27506744 0.25143078 0.23471528 0.21893318 0.2087973
 0.20123443 0.19241635 0.18675239 0.17827965 0.17192598 0.17012648
 0.1685857  0.16029835 0.15478648 0.15071778 0.16700669 0.20398846
 0.17651688 0.15795815 0.14706531 0.14086103 0.13795366 0.13627288
 0.13535147 0.13491902 0.13484367 0.13481819 0.1352735  0.23750061
 0.19492965 0.16139107 0.14873073 0.14154306 0.13746179 

In [ ]:
# plt.plot(loss_list)

In [ ]:
# l = loss_list[0]
# sum_dif = []
# for loss in loss_list:
#     dif = l-loss
#     l = loss
# #     num = np.random.randint(1, 20)
# #     print(num)

# #     if(num == 7):
# #         break
#     sum_dif = np.append(sum_dif,dif)
# print(sum_dif)
# plt.plot(sum_dif)

In [ ]:
# def show_mask(image, mask):
#     """
#     Show image and mask
#     Args:
#         image(numpyarray): The training image
#         semantic(numpyarray): The training image segmentation
#     """    
#     plt.subplot(1,2,1)
#     plt.title('image')
#     plt.imshow(image)
#     plt.subplot(1,2,2)
#     plt.title('mask')
#     plt.imshow(mask)
#     plt.show()

In [ ]:
# sample=total_data[4]
# img = sample['T1a']
# mask = sample['brainmask']

# show_mask(img.squeeze(), mask)

# img = img.unsqueeze(dim = 0)

# img = img.to(device)


# # feed it to network
# scores =  model(img)
# scores = scores.detach().cpu().squeeze().permute(1,2,0)

# scores = torch.exp(scores)

# a,b = torch.max(scores,dim=2)
# plt.imshow(b)


In [ ]:
-